In [8]:
import pandas as pd
import math

### Define DataFrame combination function

In [20]:
# calculate seconds for sent
def get_processed_df(received_data_path: str, sent_data_path: str) -> pd.DataFrame:
    """
    Get processed dataframe for a run.  
    It will merge sent and received json files and calculate seconds for sent packets.
    """
    sent_df = pd.read_json(sent_data_path)
    received_df = pd.read_json(received_data_path)
    
    sent_df = pd.json_normalize(sent_df['sent_packet_list'])
    received_df = pd.json_normalize(received_df['received_packet_list'])

    # eliminate row with packet_id = -1
    sent_df = sent_df[sent_df['packet_id'] != -1]
    received_df = received_df[received_df['packet_id'] != -1]
    sent_df['second'] = (sent_df['sent_time'] - sent_df['sent_time'].iloc[0]) / 1000000
    sent_df['second'] = sent_df['second'].apply(lambda x: math.floor(x))
    sent_df['packet_id'] = sent_df['packet_id'].astype(int)
    received_df['packet_id'] = received_df['packet_id'].astype(int)
    sent_received = pd.merge(sent_df, received_df, how='left', left_on='packet_id', right_on='packet_id', validate='one_to_one')
    sent_received['delay_in_microseconds'] = sent_received['received_time'] - sent_received['sent_time']
    return sent_received

### Define Get Status Value function

In [25]:
def get_stats_values(df: pd.DataFrame) -> None:
    total_sent_packets = df.shape[0]
    total_received_packets = df[df['received_time'].notnull()].shape[0]
    print(f"Total sent packets: {total_sent_packets}")
    print(f"Total received packets: {total_received_packets}")
    print(f"Total lost packets: {total_sent_packets - total_received_packets}")
    print(f"Total lost packets percentage: {(total_sent_packets - total_received_packets) / total_sent_packets * 100}%")
    print(f"Average delay: {df['delay_in_microseconds'].mean()} microseconds")

### Lab1

In [30]:
kb_list = [10]
drop_rate_list = [0, 5, 10, 15, 25, 50]

for kb in kb_list:
    for drop_rate in drop_rate_list:
        print(f"For {kb}KB and {drop_rate}% drop rate:")
        df = get_processed_df(f"received-data/lab1/{kb}_drop_{drop_rate}.json", f"sent-data/lab1/{kb}_drop_{drop_rate}.json")
        get_stats_values(df)
        print("--------------------")

For 10KB and 0% drop rate:
Total sent packets: 15000
Total received packets: 15000
Total lost packets: 0
Total lost packets percentage: 0.0%
Average delay: 3007.8596 microseconds
--------------------
For 10KB and 5% drop rate:
Total sent packets: 15000
Total received packets: 14972
Total lost packets: 28
Total lost packets percentage: 0.18666666666666668%
Average delay: 24802.85325941758 microseconds
--------------------
For 10KB and 10% drop rate:
Total sent packets: 15000
Total received packets: 10089
Total lost packets: 4911
Total lost packets percentage: 32.74%
Average delay: 740673.4192685103 microseconds
--------------------
For 10KB and 15% drop rate:
Total sent packets: 15000
Total received packets: 3155
Total lost packets: 11845
Total lost packets percentage: 78.96666666666667%
Average delay: 1916204.2469096673 microseconds
--------------------
For 10KB and 25% drop rate:
Total sent packets: 15000
Total received packets: 750
Total lost packets: 14250
Total lost packets percent

In [26]:
first_data = get_processed_df("./received-data/lab1/10_drop_0.json", "./sent-data/lab1/10_drop_0.json")
first_data

,packet_id,sent_time,second,received_time,delay_in_microseconds
0,0,1671713231543505,0,1671713231559453,15948
1,1,1671713231553781,0,1671713231559697,5916
2,2,1671713231564585,0,1671713231567345,2760
3,3,1671713231577960,0,1671713231581112,3152
4,4,1671713231589040,0,1671713231592058,3018
...,...,...,...,...,...
14995,14995,1671713399401714,167,1671713399404289,2575
14996,14996,1671713399412680,167,1671713399415162,2482
14997,14997,1671713399423678,167,1671713399426047,2369
14998,14998,1671713399443466,167,1671713399446392,2926


In [27]:
get_stats_values(first_data)

Total sent packets: 15000
Total received packets: 15000
Total lost packets: 0
Total lost packets percentage: 0.0%
Average delay: 3007.8596 microseconds
